In [ ]:
from gurobipy import Model, GRB, quicksum
import numpy as np

In [ ]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def build_and_solve_xor(hidden_units=3,
                        W_bound=5.0, V_bound=5.0, B_bound=5.0, C_bound=5.0,
                        pwl_range=8.0, pwl_points=21,
                        time_limit=30, verbose=True, seed=0):
    """
    MILP for XOR with one hidden ReLU layer and sigmoid output (PWL).
    Loss = L1(y - yhat). Hidden_units in {2,3,4}.
    """

    assert hidden_units in [2, 3, 4], "hidden_units must be 2, 3, or 4"

    # XOR dataset
    X = np.array([[0.,0.],
                  [0.,1.],
                  [1.,0.],
                  [1.,1.]], dtype=float)
    y = np.array([0., 1., 1., 0.], dtype=float)
    N, d = X.shape
    mH = hidden_units

    # Big-M for pre-activation splitting (ReLU)
    max_sum_abs_x = float(np.max(np.sum(np.abs(X), axis=1)))  # here = 2
    M_act = W_bound * max_sum_abs_x + B_bound  # bounds |sum_j w_ij x_j + b_i|
    # Bounds for products phi_{n,i} = p_{n,i} * v_i (need bounds on p and v)
    pL, pU = 0.0, M_act
    vL, vU = -V_bound, V_bound

    # Bound for u_n = sum_i v_i * p_{n,i} + c
    U_max = mH * V_bound * M_act + C_bound

    if verbose:
        print(f"M_act={M_act:.4g}, U_max={U_max:.4g}")

    # PWL grid for sigmoid on [-pwl_range, pwl_range]
    xs = np.linspace(-pwl_range, pwl_range, pwl_points)
    ys = sigmoid(xs)

    # Start model
    mdl = Model("xor_relu_sigmoid_milp")
    mdl.Params.Seed = seed
    mdl.Params.OutputFlag = 1 if verbose else 0
    if time_limit is not None:
        mdl.Params.TimeLimit = time_limit

    # Variables
    w = mdl.addVars(mH, d, lb=-W_bound, ub=W_bound, vtype=GRB.CONTINUOUS, name="w")
    b = mdl.addVars(mH, lb=-B_bound, ub=B_bound, vtype=GRB.CONTINUOUS, name="b")
    v = mdl.addVars(mH, lb=-V_bound, ub=V_bound, vtype=GRB.CONTINUOUS, name="v")
    c = mdl.addVar(lb=-C_bound, ub=C_bound, vtype=GRB.CONTINUOUS, name="c")

    # ReLU split variables per sample & neuron
    p = mdl.addVars(N, mH, lb=0.0, vtype=GRB.CONTINUOUS, name="p")  # positive part (ReLU output)
    q = mdl.addVars(N, mH, lb=0.0, vtype=GRB.CONTINUOUS, name="q")  # negative part
    z = mdl.addVars(N, mH, vtype=GRB.BINARY, name="z")              # gate

    # Bilinear linearization for output: phi_{n,i} = p_{n,i} * v_i
    phi = mdl.addVars(N, mH, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="phi")

    # Output pre-sigmoid and post-sigmoid
    u = mdl.addVars(N, lb=-U_max, ub=U_max, vtype=GRB.CONTINUOUS, name="u")     # pre-sigmoid
    yhat = mdl.addVars(N, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name="yhat")    # approx sigmoid(u)

    # L1 errors
    epos = mdl.addVars(N, lb=0.0, vtype=GRB.CONTINUOUS, name="eplus")
    eneg = mdl.addVars(N, lb=0.0, vtype=GRB.CONTINUOUS, name="eminus")

    # Helper: add McCormick envelope for z = x*y, with bounds [xL,xU], [yL,yU]
    def add_mccormick(zvar, xvar, yvar, xL, xU, yL, yU, tag):
        mdl.addConstr(zvar >= xL * yvar + yL * xvar - xL * yL, name=f"{tag}_lb1")
        mdl.addConstr(zvar >= xU * yvar + yU * xvar - xU * yU, name=f"{tag}_lb2")
        mdl.addConstr(zvar <= xU * yvar + yL * xvar - xU * yL, name=f"{tag}_ub1")
        mdl.addConstr(zvar <= xL * yvar + yU * xvar - xL * yU, name=f"{tag}_ub2")

    # Constraints

    # (A) Pre-activation split & ReLU gating
    # sum_j w_ij x^n_j + b_i = p_{n,i} - q_{n,i}
    for n in range(N):
        for i in range(mH):
            mdl.addConstr(
                quicksum(w[i, j] * X[n, j] for j in range(d)) + b[i] == p[n, i] - q[n, i],
                name=f"preact[n{n},i{i}]"
            )
            # gating:
            mdl.addConstr(p[n, i] <= M_act * (1 - z[n, i]), name=f"p_gate[n{n},i{i}]")
            mdl.addConstr(q[n, i] <= M_act * z[n, i],       name=f"q_gate[n{n},i{i}]")

    # (B) Output linearization: phi_{n,i} = p_{n,i} * v_i (McCormick)
    for n in range(N):
        for i in range(mH):
            add_mccormick(phi[n, i], p[n, i], v[i], pL, pU, vL, vU, tag=f"phi[n{n},i{i}]")

    # (C) Pre-sigmoid output: u_n = sum_i phi_{n,i} + c
    for n in range(N):
        mdl.addConstr(u[n] == quicksum(phi[n, i] for i in range(mH)) + c, name=f"u_def[n{n}]")

    # (D) Sigmoid via PWL: yhat_n ≈ sigmoid(u_n) on grid xs,ys
    for n in range(N):
        mdl.addGenConstrPWL(u[n], yhat[n], xs.tolist(), ys.tolist(), name=f"sig_pwl[n{n}]")

    # (E) L1 loss: y_n - yhat_n = epos_n - eneg_n
    for n in range(N):
        mdl.addConstr(y[n] - yhat[n] == epos[n] - eneg[n], name=f"abs_err[n{n}]")

    # Objective: minimize sum of absolute errors
    mdl.setObjective(quicksum(epos[n] + eneg[n] for n in range(N)), GRB.MINIMIZE)

    # Solve
    mdl.optimize()

    # Extract solution
    status = mdl.Status
    if status not in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        raise RuntimeError(f"Gurobi ended with status {status}")

    W = np.array([[w[i, j].X for j in range(d)] for i in range(mH)])
    b_vec = np.array([b[i].X for i in range(mH)])
    v_vec = np.array([v[i].X for i in range(mH)])
    c_val = c.X
    yhat_val = np.array([yhat[n].X for n in range(N)])

    if verbose:
        print("\nLearned parameters:")
        print("W =\n", W)
        print("b =", b_vec)
        print("v =", v_vec)
        print("c =", c_val)
        print("Predictions yhat =", yhat_val.round(4))
        print("Targets     y    =", y)

    return {
        "W": W, "b": b_vec, "v": v_vec, "c": c_val,
        "yhat": yhat_val, "model": mdl
    }

if __name__ == "__main__":
    # Try with 2–4 hidden neurons
    for mH in [2, 3, 4]:
        print(f"\n=== Training XOR with {mH} hidden ReLU neurons and sigmoid output ===")
        res = build_and_solve_xor(hidden_units=mH,
                                  W_bound=5.0, V_bound=5.0, B_bound=5.0, C_bound=5.0,
                                  pwl_range=8.0, pwl_points=21,
                                  time_limit=30, verbose=True, seed=1)

In [ ]:


def build_and_solve_nn_milp(X, y, l1,
                            W_bound=1.0, V_bound=1.0, B_bound=1.0,
                            time_limit=None, verbose=True, seed=0):
    """
    McCormick linearizations

    Parameters
    ----------
    X : array (N, d)
        Input data.
    y : array (N,)
        Targets.
    l1 : int
        Number of hidden units.
    W_bound, V_bound, B_bound : float
        Symmetric bounds: w_ij ∈ [-W_bound, W_bound], v_i ∈ [-V_bound, V_bound], b_i ∈ [-B_bound, B_bound].
        These bounds are required for McCormick linearizations and big-M safety.
    time_limit : float or None
        Optional solver time limit in seconds.
    verbose : bool
        Print logs and learned parameters.
    seed : int
        Random seed for solver.

    Returns
    -------
    dict with learned parameters: W (l1,d), b (l1,), v (l1,)
    """
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float)
    N, d = X.shape

    # === Big-Ms ===
    # For activation p,q gating, need M_act >= max |sum_j w_ij x^n_j + b_i|
    # Use conservative bound via norms:
    max_sum_abs_x = float(np.max(np.sum(np.abs(X), axis=1))) if N > 0 else 0.0
    M_act = W_bound * max_sum_abs_x + B_bound
    # For s_{ij} = w_{ij} v_i, use M_sparsity >= max |w * v|
    M_sparsity = W_bound * V_bound

    if verbose:
        print(f"M_act={M_act:.6g}, M_sparsity={M_sparsity:.6g}")

    m = Model("nn_milp")
    m.Params.Seed = seed
    if time_limit is not None:
        m.Params.TimeLimit = time_limit
    m.Params.OutputFlag = 1 if verbose else 0

    # === Variables ===
    # Weights and bias
    w = m.addVars(l1, d, lb=-W_bound, ub=W_bound, vtype=GRB.CONTINUOUS, name="w")
    b = m.addVars(l1, lb=-B_bound, ub=B_bound, vtype=GRB.CONTINUOUS, name="b")
    v = m.addVars(l1, lb=-V_bound, ub=V_bound, vtype=GRB.CONTINUOUS, name="v")

    # Activations split variables p, q and gating z
    p = m.addVars(N, l1, lb=0.0, vtype=GRB.CONTINUOUS, name="p")
    q = m.addVars(N, l1, lb=0.0, vtype=GRB.CONTINUOUS, name="q")
    z = m.addVars(N, l1, vtype=GRB.BINARY, name="z")

    # s_{ij} ~ w_{ij} * v_i  (McCormick) and sparsity switch t_{ij}
    s = m.addVars(l1, d, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="s")
    t = m.addVars(l1, d, vtype=GRB.BINARY, name="t")

    # phi_{n,i} ~ p_{n,i} * v_i  (for output sum_i p_i^n v_i = y^n)
    phi = m.addVars(N, l1, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="phi")

    # === Helper: add McCormick envelope for z = x * y, with bounds [xL,xU], [yL,yU] ===
    def add_mccormick(zvar, xvar, yvar, xL, xU, yL, yU, tag="mc"):
        # Lower bounds
        m.addConstr(zvar >= xL * yvar + yL * xvar - xL * yL, name=f"{tag}_lb1")
        m.addConstr(zvar >= xU * yvar + yU * xvar - xU * yU, name=f"{tag}_lb2")
        # Upper bounds
        m.addConstr(zvar <= xU * yvar + yL * xvar - xU * yL, name=f"{tag}_ub1")
        m.addConstr(zvar <= xL * yvar + yU * xvar - xL * yU, name=f"{tag}_ub2")

    # === Constraints ===

    # 1) Pre-activation split: sum_j w_ij x^n_j + b_i = p_i^n - q_i^n
    for n in range(N):
        for i in range(l1):
            m.addConstr(
                quicksum(w[i, j] * X[n, j] for j in range(d)) + b[i] == p[n, i] - q[n, i],
                name=f"preact[n{n},i{i}]"
            )

    # 2) p,q gating with z (Big-M)
    for n in range(N):
        for i in range(l1):
            m.addConstr(p[n, i] <= M_act * (1 - z[n, i]), name=f"p_gate[n{n},i{i}]")
            m.addConstr(q[n, i] <= M_act * z[n, i],     name=f"q_gate[n{n},i{i}]")

    # 3) Output: sum_i p_i^n * v_i = y^n  -> McCormick on phi_{n,i} = p_{n,i} * v_i
    pL, pU = 0.0, M_act
    vL, vU = -V_bound, V_bound
    for n in range(N):
        for i in range(l1):
            add_mccormick(phi[n, i], p[n, i], v[i], pL, pU, vL, vU, tag=f"phi[n{n},i{i}]")
        m.addConstr(quicksum(phi[n, i] for i in range(l1)) == y[n], name=f"out[n{n}]")

    # 4) s_{ij} = w_{ij} * v_i  -> McCormick
    wL, wU = -W_bound, W_bound
    for i in range(l1):
        for j in range(d):
            add_mccormick(s[i, j], w[i, j], v[i], wL, wU, vL, vU, tag=f"s[i{i},j{j}]")

    # 5) Link s_{ij} to sparsity selector t_{ij} via Big-M
    for i in range(l1):
        for j in range(d):
            m.addConstr(-M_sparsity * t[i, j] <= s[i, j], name=f"s_lb[i{i},j{j}]")
            m.addConstr( s[i, j] <=  M_sparsity * t[i, j], name=f"s_ub[i{i},j{j}]")

    # 6) Objective: minimize sum_{i,j} t_{ij}
    m.setObjective(quicksum(t[i, j] for i in range(l1) for j in range(d)), GRB.MINIMIZE)

    # === Optimize ===
    m.optimize()

    if m.Status not in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        raise RuntimeError(f"Gurobi ended with status {m.Status}")

    # Extract learned parameters
    W = np.array([[w[i, j].X for j in range(d)] for i in range(l1)])
    b_vec = np.array([b[i].X for i in range(l1)])
    v_vec = np.array([v[i].X for i in range(l1)])

    if verbose:
        print("\nLearned parameters:")
        print("W =\n", W)
        print("b =", b_vec)
        print("v =", v_vec)

    return {"W": W, "b": b_vec, "v": v_vec, "model": m}

# -------------------------- Example usage --------------------------
if __name__ == "__main__":
    # Tiny toy data: N=6, d=3
    rng = np.random.default_rng(0)
    N, d, l1 = 6, 3, 4
    X = rng.normal(size=(N, d))
    # Create a synthetic y from a hidden ReLU-ish structure (just to have targets)
    true_W = rng.uniform(-1, 1, size=(l1, d))
    true_b = rng.uniform(-0.5, 0.5, size=(l1,))
    true_v = rng.uniform(-1, 1, size=(l1,))
    pre = X @ true_W.T + true_b  # (N, l1)
    relu = np.maximum(pre, 0.0)
    y = relu @ true_v

    result = build_and_solve_nn_milp(X, y, l1,
                                     W_bound=1.0, V_bound=1.0, B_bound=1.0,
                                     time_limit=30, verbose=True, seed=42)

M_act=4.78973, M_sparsity=1
Set parameter Seed to value 42
Set parameter TimeLimit to value 30
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  30
Seed  42

Optimize a model with 246 rows, 140 columns and 696 nonzeros
Model fingerprint: 0xa375b166
Variable types: 104 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [4e-02, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-02, 5e+00]
Found heuristic solution: objective 3.0000000
Presolve time: 0.00s
Presolved: 246 rows, 140 columns, 696 nonzeros
Variable types: 104 continuous, 36 integer (36 binary)

Root relaxation: objective 0.000000e+00, 102 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 